In [1]:
import torch
import numpy as np
from mlp_mixer import MLPMixer

'''#단일 이미지로 구성된 비채 생성 (모든 픽셀값 1인 rgb 이미지)
img = torch.ones([1, 3, 224, 224]) #[배치 크기, 채널 수, 높이, 너비]

#모델 초기화
model = MLPMixer(in_channels=3, image_size=224, patch_size=16, num_classes=10,
                 dim=512, depth=5, token_dim=256, channel_dim=2048) 

#파라미터 계산
parameters = filter(lambda p: p.requires_grad, model.parameters())
parameters = sum([np.prod(p.size()) for p in parameters]) / 1_000_000
print('Trainable Parameters: %.3fM' % parameters)
#모델에 위 이미지 전달 후 out_img에 저장
out_img = model(img)

#출력 형태 -> [1,10] : 배치 크기 1에 대해 모델이 10개 클래스 각각에 대한 예측 수행
print("Shape of out :", out_img.shape)  # [B, in_channels, image_size, image_size]'''

'#단일 이미지로 구성된 비채 생성 (모든 픽셀값 1인 rgb 이미지)\nimg = torch.ones([1, 3, 224, 224]) #[배치 크기, 채널 수, 높이, 너비]\n\n#모델 초기화\nmodel = MLPMixer(in_channels=3, image_size=224, patch_size=16, num_classes=10,\n                 dim=512, depth=5, token_dim=256, channel_dim=2048) \n\n#파라미터 계산\nparameters = filter(lambda p: p.requires_grad, model.parameters())\nparameters = sum([np.prod(p.size()) for p in parameters]) / 1_000_000\nprint(\'Trainable Parameters: %.3fM\' % parameters)\n#모델에 위 이미지 전달 후 out_img에 저장\nout_img = model(img)\n\n#출력 형태 -> [1,10] : 배치 크기 1에 대해 모델이 10개 클래스 각각에 대한 예측 수행\nprint("Shape of out :", out_img.shape)  # [B, in_channels, image_size, image_size]'

DataLoader

In [2]:
#pip install scikit-learn
#!pip install numpy
#pip install 

In [3]:
import os
import pickle
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from PIL import Image

class CIFAR10Dataset(Dataset):
    def __init__(self, data_dir, transform=None, train=True):
        self.data = []
        self.labels = []
        self.transform = transform

        # 훈련 데이터 파일 또는 테스트 데이터 파일 선택
        if train:
            for i in range(1, 2):
                file_path = os.path.join(data_dir, f'data_batch_{i}')
                with open(file_path, 'rb') as f:
                    batch = pickle.load(f, encoding='bytes')
                self.data.append(batch[b'data'])
                self.labels += batch[b'labels']
        else:
            file_path = os.path.join(data_dir, 'test_batch')
            with open(file_path, 'rb') as f:
                batch = pickle.load(f, encoding='bytes')
            self.data.append(batch[b'data'])
            self.labels += batch[b'labels']
        
        self.data = np.vstack(self.data).reshape(-1, 3, 32, 32)
        self.data = self.data.transpose((0, 2, 3, 1))  # convert to HWC -> 파이토치의 형태에 맞게 전처리 적용

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img, label = self.data[idx], self.labels[idx]
        img = Image.fromarray(img)
        if self.transform:
            img = self.transform(img)
        return img, label

# 데이터 전처리 및 데이터셋 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

data_dir = 'C:/Users/hongi/Desktop/MLP-Mixer-pytorch-master/cifar-10-batches-py'
'''train_dataset = CIFAR10Dataset(data_dir=data_dir, transform=transform, train=True)
test_dataset = CIFAR10Dataset(data_dir=data_dir, transform=transform, train=False)'''

# 훈련 데이터셋 생성 후 반으로 줄이기
train_dataset_full = CIFAR10Dataset(data_dir=data_dir, transform=transform, train=True)
train_size_reduced = int(0.5 * len(train_dataset_full))  # 절반 크기
_, train_dataset = random_split(train_dataset_full, [train_size_reduced, len(train_dataset_full) - train_size_reduced])

# 테스트 데이터셋 생성 후 반으로 줄이기 (만약 필요하다면)
test_dataset_full = CIFAR10Dataset(data_dir=data_dir, transform=transform, train=False)
test_size_reduced = int(0.5 * len(test_dataset_full))  # 절반 크기
_, test_dataset = random_split(test_dataset_full, [test_size_reduced, len(test_dataset_full) - test_size_reduced])


# 훈련 데이터셋을 훈련 및 검증 데이터셋으로 분할
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])


In [4]:
print(len(train_dataset))  # 훈련 데이터셋의 크기
print(len(val_dataset))    # 검증 데이터셋의 크기
print(len(test_dataset))   # 테스트 데이터셋의 크기

first_data_point, first_label = train_dataset[0]
print(first_data_point.shape)


4000
1000
5000
torch.Size([3, 224, 224])


In [5]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [6]:
'''data_dir = 'C:/Users/hongi/Desktop/MLP-Mixer-pytorch-master/cifar-10-batches-py'  # 데이터셋 경로 수정
train_features, train_labels, test_features, test_labels = load_data(data_dir)

# 훈련 데이터를 훈련과 검증 데이터로 분할
train_features, val_features, train_labels, val_labels = train_test_split(
    train_features, train_labels, test_size=0.1, random_state=42)'''


"data_dir = 'C:/Users/hongi/Desktop/MLP-Mixer-pytorch-master/cifar-10-batches-py'  # 데이터셋 경로 수정\ntrain_features, train_labels, test_features, test_labels = load_data(data_dir)\n\n# 훈련 데이터를 훈련과 검증 데이터로 분할\ntrain_features, val_features, train_labels, val_labels = train_test_split(\n    train_features, train_labels, test_size=0.1, random_state=42)"

In [7]:
'''batch_size = 64

# 데이터를 Tensor로 변환
train_features = torch.tensor(train_features)
train_labels = torch.tensor(train_labels)
val_features = torch.tensor(val_features)
val_labels = torch.tensor(val_labels)
test_features = torch.tensor(test_features)
test_labels = torch.tensor(test_labels)

# TensorDataset 생성
train_dataset = TensorDataset(train_features, train_labels)
val_dataset = TensorDataset(val_features, val_labels)
test_dataset = TensorDataset(test_features, test_labels)

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)'''


'batch_size = 64\n\n# 데이터를 Tensor로 변환\ntrain_features = torch.tensor(train_features)\ntrain_labels = torch.tensor(train_labels)\nval_features = torch.tensor(val_features)\nval_labels = torch.tensor(val_labels)\ntest_features = torch.tensor(test_features)\ntest_labels = torch.tensor(test_labels)\n\n# TensorDataset 생성\ntrain_dataset = TensorDataset(train_features, train_labels)\nval_dataset = TensorDataset(val_features, val_labels)\ntest_dataset = TensorDataset(test_features, test_labels)\n\n# DataLoader 생성\ntrain_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)\nval_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)\ntest_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)'

model, loss func, optimizer

In [8]:
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

model = MLPMixer(in_channels=3, image_size=224, patch_size=16, num_classes=10,  # CIFAR-10 클래스 수
                 dim=512, depth=8, token_dim=256, channel_dim=2048)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.95, patience=10, verbose=True)


train

In [9]:
#pip install tqdm

In [10]:
#model.load_state_dict(torch.load('best_model_weights.pth'))


In [11]:
pip install tensorboard

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pytorch-lightning 1.6.0 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [12]:
from torch.utils.tensorboard import SummaryWriter

# 로그 파일이 저장될 디렉토리를 지정합니다.
writer = SummaryWriter('runs/my_experiment')

In [14]:
num_epochs = 1000
best_val_accuracy = 0
patience = 50
stale_epochs = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Training]')
    
    for inputs, labels in train_bar:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    # 훈련 손실 로깅
    epoch_loss = running_loss / len(train_loader)
    writer.add_scalar('Loss/train', epoch_loss, epoch)  # 수정된 위치

    print(f"Epoch {epoch+1}, Training Loss: {epoch_loss:.10f}")

    # 검증 단계
    model.eval()
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Validation]'):
            outputs = model(images)
            val_loss = criterion(outputs, labels)
            val_running_loss += val_loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(val_loader)
    val_accuracy = 100 * val_correct / val_total
    writer.add_scalar('Loss/validation', val_epoch_loss, epoch)  # 검증 손실 기록
    writer.add_scalar('Accuracy/validation', val_accuracy, epoch)  # 검증 정확도 기록

    print(f'Epoch {epoch+1}, Validation Loss: {val_epoch_loss:.10f}, Validation Accuracy: {val_accuracy:.4f}%')

    # 학습률 스케줄러 업데이트 및 early stopping 로직은 변경 없음...

writer.close()  # 텐서보드 리소스 정리
print('Finished Training')

Epoch 1/1000 [Training]: 100%|██████████| 125/125 [03:22<00:00,  1.62s/it]


Epoch 1, Training Loss: 2.1251592674


Epoch 1/1000 [Validation]: 100%|██████████| 32/32 [00:18<00:00,  1.71it/s]


Epoch 1, Validation Loss: 1.9668451026, Validation Accuracy: 26.2000%


Epoch 2/1000 [Training]: 100%|██████████| 125/125 [03:24<00:00,  1.63s/it]


...

Epoch 38, Training Loss: 0.0002579237


Epoch 38/1000 [Validation]: 100%|██████████| 32/32 [00:18<00:00,  1.70it/s]


Epoch 38, Validation Loss: 3.9889564589, Validation Accuracy: 45.4000%


Epoch 39/1000 [Training]:  94%|█████████▍| 118/125 [03:14<00:11,  1.65s/it]


KeyboardInterrupt: 

In [16]:
%load_ext tensorboard
%tensorboard --logdir runs
